In [1]:
# Process OBS so it's ready for comparison

In [46]:
import pandas as pd

In [47]:
#### Find correct trip_list_id_num for each trip 

In [48]:
# Read latest OBS data
df = pd.read_csv(r'R:\FastTrips\CHTS & OBS ft_output\OBS_ft_output_w_stops.csv')

df['person_id'] = df['person_id'].apply(lambda x: x.split("_")[-1]).astype('str')

# Fill NaN type with empty string
for field in ['A_id_num','B_id_num','A_id','B_id']:
    df[field].fillna("",inplace=True)
    
# Create a field designating the origin taz for entire path
origin = pd.DataFrame(df.groupby(['person_id','trip_list_id_num']).first()['A_id_num'])
dest = pd.DataFrame(df.groupby(['person_id','trip_list_id_num']).last()['B_id_num'])

origin['path_o_taz'] = origin['A_id_num']
origin.drop('A_id_num',axis=1,inplace=True)
origin['person_id'] = origin.index.get_level_values(0)
origin['trip_list_id_num'] = origin.index.get_level_values(1)

dest['path_d_taz'] = dest['B_id_num']
dest.drop('B_id_num',axis=1,inplace=True)
dest['person_id'] = dest.index.get_level_values(0)
dest['trip_list_id_num'] = dest.index.get_level_values(1)

# Merge to include a path_origin and path_dest field for each path link
df = pd.merge(df, origin, on=['person_id','trip_list_id_num'],how='left')
df = pd.merge(df, dest, on=['person_id','trip_list_id_num'],how='left')

In [49]:
# Read in an Fast Trips input format
obsinput = pd.read_csv('../data/obs/trip_list.txt')
obsinput['person_id'] = obsinput['person_id'].astype('int').astype('str')
obsinput['trip_list_id_num'] = obsinput['trip_list_id_num'].astype('int').astype('str')

obsinput['unique_id'] = obsinput['person_id'] + "_" + obsinput['trip_list_id_num']

In [50]:
# Drop the old trip_list_id_num and replace by merging with original input values 
df.drop('trip_list_id_num',axis=1,inplace=True)
df = pd.merge(df,obsinput[['person_id','trip_list_id_num','o_taz','d_taz']],
              left_on=['person_id','path_o_taz','path_d_taz'], right_on=['person_id','o_taz','d_taz'], 
              how='inner')    
# Using inner join returns only person_id + trip_list_id_num pairs in the actual trip_list.txt file

In [51]:
# Create unique_ids
df['unique_id'] = df['person_id'] + "_" + df['trip_list_id_num']

In [52]:
df['unique_id'].unique() == obsinput['unique_id'].unique()

False

In [10]:
len(df.groupby('unique_id').count())

22599

In [11]:
len(obsinput.groupby('unique_id').count())

26838

In [12]:
# Thinking this should match, but will continue on for now

# Transit

In [13]:
############# 
# problem: many to many join
# e.g., in OBS Napa Vine_10 has several GTFS equivalents (vine_10nb, vine_10NVT, vine_10NVTN, etc.)
# Solution for now is to use the OBS transit line fields
# Instead of attaching GTFS to the observered records, we'll attach OBS format to the model in the prepare_fastrips script

# We do want to attach agency here though

In [53]:
# Add the proper transit fields
agency_lookup = pd.read_csv('../data/obs/obs_agency_lookup.csv')

# Get a unique lookup of OBS to GTFS agency names
agency_list = agency_lookup['agency'].drop_duplicates().reset_index(drop=True)

In [54]:
len(df)

78701

In [55]:
# Join the OBS agency name
df = pd.merge(df,agency_lookup,on='route_id',how='left')

In [56]:
### Replace BART route_id specifics with generic BART 
df.ix[df['agency']=='bart', 'route_id'] = 'BART'
df.ix[df['agency']=='Caltrain', 'route_id'] = 'Caltrain'
# newmodel.ix[newmodel.agency_id=='bart', 'OBS_route_id'] = 'BART'
# newmodel.ix[newmodel.agency_id=='caltrain', 'OBS_agency'] = 'Caltrain'
# newmodel.ix[newmodel.agency_id=='caltrain', 'OBS_route_id'] = 'Caltrain'

In [57]:
df.ix[df['agency']=='BART', 'route_id'] = 'BART'

In [60]:
# Write observed records to file
df.to_csv('../data/obs/obs_links.csv')

## Later: add GTFS route ID and agency after resolving many-to-many issue

In [608]:
# Add GTFS
# obs_to_gtfs = pd.read_csv('../data/obs/obs_to_gtfs_v1.8.csv')
# obs_to_gtfs.fillna("",inplace=True)
# obs_to_gtfs.drop_duplicates(inplace=True)

# test = pd.merge(df,obs_to_gtfs,left_on='route_id',right_on='OBS_route_id',how='left')

# Join the GTFS agency name to the OBS data
# df = pd.merge(df, agency_list, left_on='agency', right_on='OBS_agency', how='left')

# Replace the OBS 
# df['agency'] = df['GTFS1.8_agency']
# df.drop(['OBS_agency','GTFS1.8_agency'],axis=1,inplace=True)

# See if we can extract any route IDs from here based on that route_id field

# rows_with_route_id = df[-df['route_id'].isnull()]
# rows_without_route_id = df[df['route_id'].isnull()]

# len(rows_with_route_id) + len(rows_without_route_id) == len(df)

# newdf = pd.merge(rows_with_route_id, obs_to_gtfs, left_on='route_id', right_on='OBS_route_id')
# output = pd.concat([newdf,rows_without_route_id])
# output[['route_id','agency']].fillna("",inplace=True)
# output.sort(['person_id','trip_list_id_num']).to_csv('../data/obs/obs_links.csv')